In [10]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from math import *

%matplotlib inline

In [30]:
def backtest(data):
    len_time = len(data.rows)
    
    slippage = 0.0005
    leverage = 2
    
    PNL = np.zeros(len_time-60)
    PNL[0] = 100
    
    positions = np.zeros(len(pairs))
    new_positions = np.zeros(len(pairs))
    beta_array = np.zeros(len(pairs))
    
    
    trading_pairs = [pairs, positions]    
    for i in range(len(pairs)):
        priceA = data[pairs[i][0]][0:59]
        priceB = data[pairs[i][1]][0:59]
            
        signal = generate_signal(priceA, priceB)
        beta[i] = signal[0]
        signal = signal[1]
            
        if signal == 'BUYOPEN':
            positions[i] = 1
        elif signal == 'SELLOPEN':
            positions[i] = -1
        else:
            positions[i] = 0
            
    invest_per_pair = PNL[0] * leverage / sum(positions)
    positions = positions * invest_per_pair
        
    for t in range(61,len_time):
        PNL_part = 0
        for i in range(len(pairs)):
            priceA = data[pairs[i][0]][t-60:t-1]
            priceB = data[pairs[i][1]][t-60:t-1]
            
            A_prev = priceA[-1]
            B_prev = priceB[-1]
            A_current = data[pairs[i][0]][t]
            B_current = data[pairs[i][1]][t]
            
            A_shares = positions[i]/(A_prev - beta[i]*B_prev)
            B_shares = -beta[i] * A_shares
            
            PNL_part += A_shares * (A_current - A_prev) + B_shares * (B_current - B_prev)
            
            signal = generate_signal(priceA, priceB)
            beta[i] = signal[0]
            signal = signal[1]
            
            if signal == 'BUYOPEN':
                new_positions[i] = 1
            elif signal == 'SELLOPEN':
                new_positions[i] = -1
            else:
                new_positions[i] = 0
        
        invest_per_pair = PNL_array[0] * leverage / sum(new_positions)
        new_positions = new_positions * invest_per_pair
        PNL[t-60] = PNL_part + PNL[t-61] * (1 + r) - sum(positions) * r - sum(abs(new_positions - positions)) * slippage
        positions = new_positions 
        
    return PNL
            